In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from desispec.io.spectra import read_spectra
from desispec.coaddition import coadd_cameras
import numpy as np
from desitarget.targetmask import desi_mask
from desispec.resolution import Resolution
from scipy.optimize import curve_fit


In [ ]:
spec = read_spectra("coadd-main-dark-0-test.fits")
spec0 = spec[1]
print(spec[1])
wave = coadd_cameras(spec0).wave['brz']
res = coadd_cameras(spec0).resolution_data['brz'][0]
R = Resolution(res)
# print(R)

In [ ]:
sigArray = []
rArray = []
def gauss(x,sig,A,mu):
    return A*np.exp(-((x-mu)**2) / (2*sig**2))

for i in range(len(wave)):
    kernel = res[:, i]  # shape: (ndiag,), centered at wave[i]
    # Compute the pixel offsets relative to center
    offsets = R.offsets
    kernel_wave = wave[i] + (offsets * np.gradient(wave)[i])
    if np.nan in kernel:
        sigArray.append(np.nan)
        rArray.append(np.nan)
    elif np.inf in kernel:
        sigArray.append(np.inf)
        rArray.append(np.inf)
    else:
        # Get the resolution kernel (ISF) at pixel i
        kernel = res[:, i]  # shape: (ndiag,), centered at wave[i]

        # Compute the pixel offsets relative to center
        offsets = R.offsets
        kernel_wave = wave[i] + (offsets * np.gradient(wave)[i])
        peak = max(kernel)

        parameters = curve_fit(gauss, kernel_wave, kernel, p0=[0.5, 1, kernel_wave[5]])
        sig_fit, A_fit, mu_fit = parameters[0]
        sigArray.append(sig_fit)
        print(sig_fit)
        r = kernel_wave[5]/(2*sig_fit*np.sqrt(2*np.log(2)))
        rArray.append(r)

        # # Normalize (optional; usually already normalized)
        # kernel /= np.sum(kernel)

In [ ]:
plt.figure(1)
plt.plot(wave,rArray,'b.')
plt.xlabel('$\lambda\ [{\AA}]$')
plt.ylabel('Resolution')


plt.figure(2)
plt.plot(wave,sigArray,'k.')
plt.xlabel('$\lambda\ [{\AA}]$')
plt.ylabel('$\Delta\sigma_{s}\ [Pixels]$');

